# Assignment 2: Homography Estimation

**Author:** [Your Student Number]  
**Date:** November 6, 2025

## Table of Contents
1. [Setup and Imports](#setup)
2. [Part 1: Feature Extraction](#part1)
3. [Part 2: Feature Matching](#part2)
4. [Part 3: Homography Estimation (DLT + RANSAC)](#part3)
5. [Part 4: Image Warping and Panorama Construction](#part4)
6. [Augmented Reality Application](#ar)
7. [Results and Visualizations](#results)

## Setup and Imports <a name="setup"></a>

Install required libraries and import necessary packages.

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib backend for better display
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

# Print versions
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

# Define paths
BASE_PATH = Path("/Users/otisvaliant/Desktop/Homography-Estimation")
PANORAMA_PATH = BASE_PATH / "panorama_dataset"
AR_PATH = BASE_PATH / "ar_dataset"
OUTPUT_PATH = BASE_PATH / "panorama_results"

# Create output directories
OUTPUT_PATH.mkdir(exist_ok=True)
(OUTPUT_PATH / "visualizations").mkdir(exist_ok=True)

print("\nDataset paths:")
print(f"Panorama dataset: {PANORAMA_PATH}")
print(f"AR dataset: {AR_PATH}")
print(f"Output path: {OUTPUT_PATH}")

# List available scenes
scenes = [d.name for d in PANORAMA_PATH.iterdir() if d.is_dir()]
print(f"\nAvailable scenes: {scenes}")

## Part 1: Feature Extraction <a name="part1"></a>

Implement SIFT, SURF, and ORB feature detectors and descriptors. We'll create a unified interface for all three methods.

In [ ]:
class FeatureExtractor:
    """
    Unified interface for SIFT, SURF, and ORB feature detection and description.
    
    Properties:
    - SIFT: Scale-Invariant Feature Transform - robust to scale and rotation, uses 128-dim descriptors
    - SURF: Speeded Up Robust Features - faster than SIFT, uses 64 or 128-dim descriptors
    - ORB: Oriented FAST and Rotated BRIEF - very fast, binary descriptors (256 bits)
    """
    
    def __init__(self, method: str = 'SIFT'):
        """
        Initialize feature extractor with specified method.
        
        Args:
            method: 'SIFT', 'SURF', or 'ORB'
        """
        self.method = method.upper()
        
        if self.method == 'SIFT':
            self.detector = cv2.SIFT_create()
            self.descriptor_type = 'float'
        elif self.method == 'SURF':
            # SURF requires opencv-contrib-python
            try:
                self.detector = cv2.xfeatures2d.SURF_create()
                self.descriptor_type = 'float'
            except AttributeError:
                print("SURF not available. Install opencv-contrib-python or use SIFT/ORB")
                raise
        elif self.method == 'ORB':
            self.detector = cv2.ORB_create(nfeatures=5000)  # Increase max features for better matching
            self.descriptor_type = 'binary'
        else:
            raise ValueError(f"Unknown method: {method}. Use 'SIFT', 'SURF', or 'ORB'")
    
    def detect_and_compute(self, image: np.ndarray) -> Tuple[List, np.ndarray]:
        """
        Detect keypoints and compute descriptors.
        
        Args:
            image: Input image (grayscale or color)
            
        Returns:
            keypoints: List of cv2.KeyPoint objects
            descriptors: NumPy array of descriptors
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        # Detect and compute
        keypoints, descriptors = self.detector.detectAndCompute(gray, None)
        
        return keypoints, descriptors
    
    def visualize_keypoints(self, image: np.ndarray, keypoints: List, 
                           title: str = "Detected Keypoints") -> np.ndarray:
        """
        Visualize detected keypoints on image.
        
        Args:
            image: Input image
            keypoints: List of cv2.KeyPoint objects
            title: Plot title
            
        Returns:
            Image with drawn keypoints
        """
        img_kp = cv2.drawKeypoints(image, keypoints, None, 
                                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
                                    color=(0, 255, 0))
        
        plt.figure(figsize=(15, 10))
        plt.imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
        plt.title(f"{title} - {self.method} ({len(keypoints)} keypoints)")
        plt.axis('off')
        plt.tight_layout()
        
        return img_kp

print("FeatureExtractor class defined successfully!")

## Part 2: Feature Matching <a name="part2"></a>

Implement feature matching using k-NN with Lowe's ratio test for filtering ambiguous matches.

In [ ]:
class FeatureMatcher:
    """
    Feature matcher using k-NN with Lowe's ratio test.
    
    Lowe's ratio test: A match is considered good if the distance to the nearest neighbor
    is significantly smaller than the distance to the second nearest neighbor.
    Typical threshold: 0.7-0.8
    """
    
    def __init__(self, descriptor_type: str = 'float', ratio_threshold: float = 0.75, 
                 cross_check: bool = False):
        """
        Initialize feature matcher.
        
        Args:
            descriptor_type: 'float' for SIFT/SURF, 'binary' for ORB
            ratio_threshold: Lowe's ratio test threshold (typically 0.7-0.8)
            cross_check: Whether to perform cross-check (match must be mutual)
        """
        self.ratio_threshold = ratio_threshold
        self.cross_check = cross_check
        
        # Choose appropriate matcher based on descriptor type
        if descriptor_type == 'float':
            # For SIFT/SURF: Use FLANN with KDTree
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            search_params = dict(checks=50)
            self.matcher = cv2.FlannBasedMatcher(index_params, search_params)
        else:
            # For ORB: Use FLANN with LSH or BruteForce with Hamming distance
            FLANN_INDEX_LSH = 6
            index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, 
                               key_size=12, multi_probe_level=1)
            search_params = dict(checks=50)
            self.matcher = cv2.FlannBasedMatcher(index_params, search_params)
    
    def match(self, descriptors1: np.ndarray, descriptors2: np.ndarray) -> List[cv2.DMatch]:
        """
        Match features between two images using k-NN and Lowe's ratio test.
        
        Args:
            descriptors1: Descriptors from first image
            descriptors2: Descriptors from second image
            
        Returns:
            List of good matches (cv2.DMatch objects)
        """
        if descriptors1 is None or descriptors2 is None:
            return []
        
        # Ensure descriptors are in correct format
        if descriptors1.dtype != np.float32:
            descriptors1 = descriptors1.astype(np.float32)
        if descriptors2.dtype != np.float32:
            descriptors2 = descriptors2.astype(np.float32)
        
        # Find k=2 nearest neighbors
        matches = self.matcher.knnMatch(descriptors1, descriptors2, k=2)
        
        # Apply Lowe's ratio test
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < self.ratio_threshold * n.distance:
                    good_matches.append(m)
        
        # Optional: Cross-check
        if self.cross_check and len(good_matches) > 0:
            good_matches = self._cross_check_matches(descriptors1, descriptors2, good_matches)
        
        return good_matches
    
    def _cross_check_matches(self, desc1: np.ndarray, desc2: np.ndarray, 
                            matches: List[cv2.DMatch]) -> List[cv2.DMatch]:
        """
        Perform cross-check: keep only matches that are mutual best matches.
        """
        # Match in reverse direction
        matches_reverse = self.matcher.knnMatch(desc2, desc1, k=2)
        
        # Build set of reverse matches
        reverse_match_set = set()
        for match_pair in matches_reverse:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < self.ratio_threshold * n.distance:
                    reverse_match_set.add((m.queryIdx, m.trainIdx))
        
        # Keep only mutual matches
        cross_checked = []
        for m in matches:
            if (m.trainIdx, m.queryIdx) in reverse_match_set:
                cross_checked.append(m)
        
        return cross_checked
    
    def visualize_matches(self, img1: np.ndarray, kp1: List, 
                         img2: np.ndarray, kp2: List, 
                         matches: List[cv2.DMatch], 
                         title: str = "Feature Matches",
                         max_display: int = 100) -> np.ndarray:
        """
        Visualize feature matches between two images.
        
        Args:
            img1, img2: Input images
            kp1, kp2: Keypoints
            matches: List of matches
            title: Plot title
            max_display: Maximum number of matches to display
            
        Returns:
            Image with drawn matches
        """
        # Limit matches for visualization
        matches_to_draw = matches[:max_display] if len(matches) > max_display else matches
        
        img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches_to_draw, None,
                                      flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        plt.figure(figsize=(20, 10))
        plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
        plt.title(f"{title} ({len(matches)} total matches, showing {len(matches_to_draw)})")
        plt.axis('off')
        plt.tight_layout()
        
        return img_matches

print("FeatureMatcher class defined successfully!")

## Part 3: Homography Estimation - DLT + RANSAC <a name="part3"></a>

Manual implementation of Direct Linear Transform (DLT) and RANSAC algorithm for robust homography estimation.

### Theory:
- **DLT**: Solves for homography matrix H (3×3) using point correspondences
- **Normalization**: Improves numerical stability by normalizing coordinates
- **RANSAC**: Random Sample Consensus - iteratively fits models and finds inliers

In [ ]:
class HomographyEstimator:
    """
    Manual implementation of homography estimation using DLT and RANSAC.
    
    The homography H maps points from image 1 to image 2:
    x2 = H @ x1 (in homogeneous coordinates)
    
    DLT minimizes ||Ah||^2 subject to ||h|| = 1, where h is the vectorized H.
    """
    
    def __init__(self, ransac_iterations: int = 2000, 
                 ransac_threshold: float = 5.0,
                 normalize: bool = True):
        """
        Initialize homography estimator.
        
        Args:
            ransac_iterations: Number of RANSAC iterations
            ransac_threshold: Inlier threshold in pixels
            normalize: Whether to normalize point coordinates
        """
        self.ransac_iterations = ransac_iterations
        self.ransac_threshold = ransac_threshold
        self.normalize = normalize
    
    def normalize_points(self, points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Normalize points for numerical stability.
        
        Transforms points so that:
        - Centroid is at origin
        - Average distance from origin is sqrt(2)
        
        Args:
            points: Nx2 array of points
            
        Returns:
            normalized_points: Nx2 normalized points
            T: 3x3 normalization transformation matrix
        """
        # Compute centroid
        centroid = np.mean(points, axis=0)
        
        # Center points
        centered = points - centroid
        
        # Compute average distance from origin
        avg_dist = np.mean(np.sqrt(np.sum(centered**2, axis=1)))
        
        # Scale factor to make average distance sqrt(2)
        if avg_dist > 0:
            scale = np.sqrt(2) / avg_dist
        else:
            scale = 1.0
        
        # Normalization matrix
        T = np.array([
            [scale, 0, -scale * centroid[0]],
            [0, scale, -scale * centroid[1]],
            [0, 0, 1]
        ])
        
        # Apply normalization
        normalized = centered * scale
        
        return normalized, T
    
    def compute_homography_dlt(self, src_pts: np.ndarray, dst_pts: np.ndarray) -> np.ndarray:
        """
        Compute homography using Direct Linear Transform (DLT).
        
        For each correspondence (x, y) -> (x', y'), we have:
        x' = (h11*x + h12*y + h13) / (h31*x + h32*y + h33)
        y' = (h21*x + h22*y + h23) / (h31*x + h32*y + h33)
        
        This gives us 2 equations per correspondence.
        We need at least 4 correspondences (8 equations) to solve for 8 unknowns.
        
        Args:
            src_pts: Nx2 array of source points
            dst_pts: Nx2 array of destination points
            
        Returns:
            H: 3x3 homography matrix
        """
        n_points = src_pts.shape[0]
        
        if n_points < 4:
            raise ValueError("At least 4 point correspondences are required")
        
        # Normalize points if requested
        if self.normalize:
            src_norm, T_src = self.normalize_points(src_pts)
            dst_norm, T_dst = self.normalize_points(dst_pts)
        else:
            src_norm = src_pts
            dst_norm = dst_pts
        
        # Build matrix A (2n x 9)
        A = []
        for i in range(n_points):
            x, y = src_norm[i]
            xp, yp = dst_norm[i]
            
            # Two rows per correspondence
            A.append([-x, -y, -1, 0, 0, 0, x*xp, y*xp, xp])
            A.append([0, 0, 0, -x, -y, -1, x*yp, y*yp, yp])
        
        A = np.array(A)
        
        # Solve using SVD: A = U S V^T
        # Solution is the last column of V (corresponds to smallest singular value)
        U, S, Vt = np.linalg.svd(A)
        h = Vt[-1, :]
        
        # Reshape to 3x3 matrix
        H = h.reshape(3, 3)
        
        # Denormalize if normalization was applied
        if self.normalize:
            # H = T_dst^-1 @ H_norm @ T_src
            H = np.linalg.inv(T_dst) @ H @ T_src
        
        # Normalize so that H[2,2] = 1
        H = H / H[2, 2]
        
        return H
    
    def compute_reprojection_error(self, src_pts: np.ndarray, dst_pts: np.ndarray, 
                                   H: np.ndarray) -> np.ndarray:
        """
        Compute reprojection error for each point.
        
        Args:
            src_pts: Nx2 source points
            dst_pts: Nx2 destination points
            H: 3x3 homography matrix
            
        Returns:
            errors: N array of reprojection errors (Euclidean distance)
        """
        n_points = src_pts.shape[0]
        
        # Convert to homogeneous coordinates
        src_h = np.hstack([src_pts, np.ones((n_points, 1))])
        
        # Project points using H
        projected_h = (H @ src_h.T).T
        
        # Convert back to Euclidean coordinates
        projected = projected_h[:, :2] / projected_h[:, 2:3]
        
        # Compute Euclidean distance
        errors = np.sqrt(np.sum((projected - dst_pts)**2, axis=1))
        
        return errors
    
    def estimate_homography_ransac(self, src_pts: np.ndarray, dst_pts: np.ndarray) -> Tuple:
        """
        Estimate homography using RANSAC.
        
        Algorithm:
        1. Randomly select 4 correspondences
        2. Compute homography using DLT
        3. Count inliers (points with reprojection error < threshold)
        4. Repeat for N iterations
        5. Return homography with most inliers
        
        Args:
            src_pts: Nx2 source points
            dst_pts: Nx2 destination points
            
        Returns:
            best_H: 3x3 homography matrix
            inlier_mask: Boolean mask indicating inliers
            inlier_ratio: Ratio of inliers to total points
        """
        n_points = src_pts.shape[0]
        
        if n_points < 4:
            raise ValueError("At least 4 point correspondences are required")
        
        best_H = None
        best_inliers = None
        best_inlier_count = 0
        
        np.random.seed(42)  # For reproducibility
        
        for iteration in range(self.ransac_iterations):
            # Randomly select 4 points
            indices = np.random.choice(n_points, 4, replace=False)
            sample_src = src_pts[indices]
            sample_dst = dst_pts[indices]
            
            try:
                # Compute homography from sample
                H = self.compute_homography_dlt(sample_src, sample_dst)
                
                # Compute reprojection errors for all points
                errors = self.compute_reprojection_error(src_pts, dst_pts, H)
                
                # Identify inliers
                inliers = errors < self.ransac_threshold
                inlier_count = np.sum(inliers)
                
                # Update best model if this is better
                if inlier_count > best_inlier_count:
                    best_inlier_count = inlier_count
                    best_inliers = inliers
                    best_H = H
                    
            except np.linalg.LinAlgError:
                # Skip degenerate configurations
                continue
        
        if best_H is None:
            raise RuntimeError("RANSAC failed to find a valid homography")
        
        # Refine homography using all inliers
        inlier_src = src_pts[best_inliers]
        inlier_dst = dst_pts[best_inliers]
        best_H = self.compute_homography_dlt(inlier_src, inlier_dst)
        
        # Recompute inliers with refined H
        errors = self.compute_reprojection_error(src_pts, dst_pts, best_H)
        best_inliers = errors < self.ransac_threshold
        
        inlier_ratio = np.sum(best_inliers) / n_points
        
        return best_H, best_inliers, inlier_ratio
    
    def visualize_inliers(self, img1: np.ndarray, kp1: List, 
                         img2: np.ndarray, kp2: List,
                         matches: List[cv2.DMatch], 
                         inlier_mask: np.ndarray,
                         title: str = "Inliers vs Outliers") -> np.ndarray:
        """
        Visualize inlier and outlier matches.
        
        Args:
            img1, img2: Input images
            kp1, kp2: Keypoints
            matches: List of matches
            inlier_mask: Boolean array indicating inliers
            title: Plot title
            
        Returns:
            Visualization image
        """
        # Separate inliers and outliers
        inlier_matches = [m for i, m in enumerate(matches) if inlier_mask[i]]
        outlier_matches = [m for i, m in enumerate(matches) if not inlier_mask[i]]
        
        # Draw outliers in red
        img_outliers = cv2.drawMatches(img1, kp1, img2, kp2, outlier_matches, None,
                                       matchColor=(255, 0, 0),
                                       flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        # Draw inliers in green on top
        img_result = cv2.drawMatches(img1, kp1, img2, kp2, inlier_matches, img_outliers,
                                     matchColor=(0, 255, 0),
                                     flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        plt.figure(figsize=(20, 10))
        plt.imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
        n_inliers = len(inlier_matches)
        n_total = len(matches)
        plt.title(f"{title} - Inliers: {n_inliers}/{n_total} ({100*n_inliers/n_total:.1f}%)")
        plt.axis('off')
        plt.tight_layout()
        
        return img_result

print("HomographyEstimator class defined successfully!")

## Part 4: Image Warping and Panorama Construction <a name="part4"></a>

Implement image warping using homography and create panoramas with blending.

In [ ]:
class PanoramaStitcher:
    """
    Image warping and panorama stitching using homography.
    """
    
    def __init__(self, blend_method: str = 'linear'):
        """
        Initialize panorama stitcher.
        
        Args:
            blend_method: 'copy', 'average', or 'linear' (feathering)
        """
        self.blend_method = blend_method
    
    def warp_image(self, img: np.ndarray, H: np.ndarray, 
                   output_shape: Optional[Tuple[int, int]] = None) -> Tuple[np.ndarray, Tuple]:
        """
        Warp image using homography matrix.
        
        Args:
            img: Input image
            H: 3x3 homography matrix
            output_shape: Optional (height, width) for output canvas
            
        Returns:
            warped: Warped image
            offset: (x_offset, y_offset) to handle negative coordinates
        """
        h, w = img.shape[:2]
        
        # Find corners of source image
        corners = np.array([
            [0, 0],
            [w-1, 0],
            [w-1, h-1],
            [0, h-1]
        ], dtype=np.float32)
        
        # Transform corners to destination
        corners_h = np.hstack([corners, np.ones((4, 1))])
        corners_transformed = (H @ corners_h.T).T
        corners_transformed = corners_transformed[:, :2] / corners_transformed[:, 2:3]
        
        # Find bounding box
        min_x = np.floor(corners_transformed[:, 0].min()).astype(int)
        max_x = np.ceil(corners_transformed[:, 0].max()).astype(int)
        min_y = np.floor(corners_transformed[:, 1].min()).astype(int)
        max_y = np.ceil(corners_transformed[:, 1].max()).astype(int)
        
        # Create translation matrix to handle negative coordinates
        offset = (-min_x, -min_y)
        T = np.array([
            [1, 0, offset[0]],
            [0, 1, offset[1]],
            [0, 0, 1]
        ])
        
        # Combined transformation
        H_translated = T @ H
        
        # Determine output size
        if output_shape is None:
            out_w = max_x - min_x
            out_h = max_y - min_y
        else:
            out_h, out_w = output_shape
        
        # Warp image
        warped = cv2.warpPerspective(img, H_translated, (out_w, out_h))
        
        return warped, offset
    
    def create_panorama_pair(self, img1: np.ndarray, img2: np.ndarray, 
                            H: np.ndarray) -> np.ndarray:
        """
        Create panorama from two images.
        
        Args:
            img1: Reference image
            img2: Image to be warped onto img1's plane
            H: Homography mapping img2 to img1
            
        Returns:
            panorama: Stitched panorama image
        """
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        
        # Find corners of both images in the panorama coordinate system
        corners1 = np.array([
            [0, 0],
            [w1, 0],
            [w1, h1],
            [0, h1]
        ], dtype=np.float32)
        
        corners2 = np.array([
            [0, 0],
            [w2, 0],
            [w2, h2],
            [0, h2]
        ], dtype=np.float32)
        
        # Transform corners2 using H
        corners2_h = np.hstack([corners2, np.ones((4, 1))])
        corners2_transformed = (H @ corners2_h.T).T
        corners2_transformed = corners2_transformed[:, :2] / corners2_transformed[:, 2:3]
        
        # Combine all corners
        all_corners = np.vstack([corners1, corners2_transformed])
        
        # Find bounding box
        min_x = np.floor(all_corners[:, 0].min()).astype(int)
        max_x = np.ceil(all_corners[:, 0].max()).astype(int)
        min_y = np.floor(all_corners[:, 1].min()).astype(int)
        max_y = np.ceil(all_corners[:, 1].max()).astype(int)
        
        # Translation to handle negative coordinates
        offset = (-min_x, -min_y)
        T = np.array([
            [1, 0, offset[0]],
            [0, 1, offset[1]],
            [0, 0, 1]
        ])
        
        # Output size
        out_w = max_x - min_x
        out_h = max_y - min_y
        
        # Warp img2
        warped2 = cv2.warpPerspective(img2, T @ H, (out_w, out_h))
        
        # Warp img1 (just translation)
        warped1 = cv2.warpPerspective(img1, T, (out_w, out_h))
        
        # Blend images
        panorama = self.blend_images(warped1, warped2)
        
        return panorama
    
    def blend_images(self, img1: np.ndarray, img2: np.ndarray) -> np.ndarray:
        """
        Blend two images based on selected method.
        
        Args:
            img1: First warped image
            img2: Second warped image
            
        Returns:
            blended: Blended result
        """
        # Create masks for non-zero regions
        mask1 = (np.sum(img1, axis=2) > 0).astype(np.float32)
        mask2 = (np.sum(img2, axis=2) > 0).astype(np.float32)
        
        # Find overlap region
        overlap = np.logical_and(mask1, mask2).astype(np.float32)
        
        if self.blend_method == 'copy':
            # Simple copy: img2 where available, else img1
            result = img1.copy()
            result[mask2 > 0] = img2[mask2 > 0]
            
        elif self.blend_method == 'average':
            # Average in overlap region
            result = img1.copy()
            overlap_mask = overlap > 0
            result[overlap_mask] = (img1[overlap_mask] + img2[overlap_mask]) // 2
            result[np.logical_and(mask2 > 0, mask1 == 0)] = img2[np.logical_and(mask2 > 0, mask1 == 0)]
            
        elif self.blend_method == 'linear':
            # Linear blending (feathering) in overlap region
            # Create distance transforms
            import scipy.ndimage as ndimage
            
            dist1 = ndimage.distance_transform_edt(mask1)
            dist2 = ndimage.distance_transform_edt(mask2)
            
            # Normalize weights in overlap region
            weights1 = dist1 / (dist1 + dist2 + 1e-10)
            weights2 = dist2 / (dist1 + dist2 + 1e-10)
            
            # Expand dimensions for broadcasting
            weights1 = weights1[:, :, np.newaxis]
            weights2 = weights2[:, :, np.newaxis]
            
            # Blend
            result = (weights2 * img1 + weights1 * img2).astype(np.uint8)
            
            # Fill non-overlap regions
            result[np.logical_and(mask1 > 0, mask2 == 0)] = img1[np.logical_and(mask1 > 0, mask2 == 0)]
            result[np.logical_and(mask2 > 0, mask1 == 0)] = img2[np.logical_and(mask2 > 0, mask1 == 0)]
        
        else:
            raise ValueError(f"Unknown blend method: {self.blend_method}")
        
        return result
    
    def create_multi_image_panorama(self, images: List[np.ndarray], 
                                   homographies: List[np.ndarray]) -> np.ndarray:
        """
        Create panorama from multiple images.
        
        Args:
            images: List of images (first is reference)
            homographies: List of homographies mapping each image to reference
                         (should have len(images)-1 entries)
            
        Returns:
            panorama: Stitched panorama
        """
        if len(images) < 2:
            return images[0] if len(images) == 1 else None
        
        # Start with reference image
        panorama = images[0]
        
        # Stitch each image sequentially
        for i, (img, H) in enumerate(zip(images[1:], homographies)):
            print(f"Stitching image {i+2}/{len(images)}...")
            panorama = self.create_panorama_pair(panorama, img, H)
        
        return panorama

print("PanoramaStitcher class defined successfully!")

## Complete Pipeline: Test on Sample Scene <a name="test"></a>

Let's test the complete pipeline on a sample scene before running on all datasets.

In [ ]:
# Test pipeline on v_bird scene with images 1 and 2
test_scene = "v_bird"
scene_path = PANORAMA_PATH / test_scene

# Load images
img1 = cv2.imread(str(scene_path / "1.png"))
img2 = cv2.imread(str(scene_path / "2.png"))

print(f"Testing pipeline on scene: {test_scene}")
print(f"Image 1 shape: {img1.shape}")
print(f"Image 2 shape: {img2.shape}")

# Test with SIFT
print("\n=== Testing with SIFT ===")
extractor = FeatureExtractor(method='SIFT')
kp1, desc1 = extractor.detect_and_compute(img1)
kp2, desc2 = extractor.detect_and_compute(img2)
print(f"Keypoints detected - Image 1: {len(kp1)}, Image 2: {len(kp2)}")

# Visualize keypoints
_ = extractor.visualize_keypoints(img1, kp1, f"{test_scene} - Image 1 Keypoints")
plt.savefig(OUTPUT_PATH / "visualizations" / f"{test_scene}_img1_keypoints.png", dpi=150, bbox_inches='tight')
plt.show()

_ = extractor.visualize_keypoints(img2, kp2, f"{test_scene} - Image 2 Keypoints")
plt.savefig(OUTPUT_PATH / "visualizations" / f"{test_scene}_img2_keypoints.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Feature matching
matcher = FeatureMatcher(descriptor_type='float', ratio_threshold=0.75)
matches = matcher.match(desc1, desc2)
print(f"\nMatches found: {len(matches)}")

# Visualize matches
_ = matcher.visualize_matches(img1, kp1, img2, kp2, matches, 
                              f"{test_scene} - Feature Matches", max_display=100)
plt.savefig(OUTPUT_PATH / "visualizations" / f"{test_scene}_matches.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Extract matched point coordinates
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])

# Homography estimation with RANSAC
estimator = HomographyEstimator(ransac_iterations=2000, ransac_threshold=5.0)
H, inlier_mask, inlier_ratio = estimator.estimate_homography_ransac(src_pts, dst_pts)

print(f"\nHomography estimation:")
print(f"Inliers: {np.sum(inlier_mask)}/{len(matches)} ({inlier_ratio*100:.2f}%)")
print(f"Homography matrix:\n{H}")

# Visualize inliers vs outliers
_ = estimator.visualize_inliers(img1, kp1, img2, kp2, matches, inlier_mask,
                                 f"{test_scene} - Inliers vs Outliers")
plt.savefig(OUTPUT_PATH / "visualizations" / f"{test_scene}_inliers.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Create panorama
stitcher = PanoramaStitcher(blend_method='linear')
panorama = stitcher.create_panorama_pair(img1, img2, H)

print(f"\nPanorama created with shape: {panorama.shape}")

# Visualize panorama
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
plt.title(f"{test_scene} - Panorama")
plt.axis('off')
plt.tight_layout()
plt.savefig(OUTPUT_PATH / f"{test_scene}_panorama_test.png", dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Pipeline test completed successfully!")

## Process All Scenes - Create Panoramas <a name="all-scenes"></a>

Now let's process all 6 scenes and create panoramas for each.

In [ ]:
def process_scene(scene_name: str, method: str = 'SIFT', num_images: int = 6):
    """
    Process a complete scene and create panorama.
    
    Args:
        scene_name: Name of the scene folder
        method: Feature detection method
        num_images: Number of images in the scene
        
    Returns:
        results: Dictionary with panorama and statistics
    """
    print(f"\n{'='*60}")
    print(f"Processing scene: {scene_name} with {method}")
    print(f"{'='*60}")
    
    scene_path = PANORAMA_PATH / scene_name
    
    # Load all images
    images = []
    for i in range(1, num_images + 1):
        img_path = scene_path / f"{i}.png"
        if img_path.exists():
            img = cv2.imread(str(img_path))
            images.append(img)
        else:
            print(f"Warning: {img_path} not found")
    
    if len(images) < 2:
        print(f"Insufficient images in {scene_name}")
        return None
    
    print(f"Loaded {len(images)} images")
    
    # Initialize components
    extractor = FeatureExtractor(method=method)
    matcher = FeatureMatcher(descriptor_type=extractor.descriptor_type, ratio_threshold=0.75)
    estimator = HomographyEstimator(ransac_iterations=2000, ransac_threshold=5.0)
    stitcher = PanoramaStitcher(blend_method='linear')
    
    # Extract features for all images
    keypoints = []
    descriptors = []
    for i, img in enumerate(images):
        kp, desc = extractor.detect_and_compute(img)
        keypoints.append(kp)
        descriptors.append(desc)
        print(f"Image {i+1}: {len(kp)} keypoints")
    
    # Compute homographies relative to first image
    homographies = []
    statistics = []
    
    for i in range(1, len(images)):
        print(f"\nEstimating homography: Image {i+1} -> Image 1")
        
        # Match features
        matches = matcher.match(descriptors[i], descriptors[0])
        print(f"  Matches: {len(matches)}")
        
        if len(matches) < 4:
            print(f"  Warning: Insufficient matches for homography estimation")
            continue
        
        # Extract point coordinates
        src_pts = np.float32([keypoints[i][m.queryIdx].pt for m in matches])
        dst_pts = np.float32([keypoints[0][m.trainIdx].pt for m in matches])
        
        # Estimate homography
        H, inlier_mask, inlier_ratio = estimator.estimate_homography_ransac(src_pts, dst_pts)
        homographies.append(H)
        
        # Compute reprojection error
        errors = estimator.compute_reprojection_error(src_pts[inlier_mask], 
                                                      dst_pts[inlier_mask], H)
        avg_error = np.mean(errors)
        
        stats = {
            'image_pair': f"{i+1}->1",
            'matches': len(matches),
            'inliers': np.sum(inlier_mask),
            'inlier_ratio': inlier_ratio,
            'avg_reprojection_error': avg_error
        }
        statistics.append(stats)
        
        print(f"  Inliers: {stats['inliers']}/{stats['matches']} ({inlier_ratio*100:.2f}%)")
        print(f"  Avg reprojection error: {avg_error:.3f} pixels")
    
    # Create panorama
    print(f"\nCreating panorama...")
    if len(homographies) > 0:
        # Stitch images sequentially
        panorama = images[0]
        for i, H in enumerate(homographies):
            print(f"  Stitching image {i+2}...")
            panorama = stitcher.create_panorama_pair(panorama, images[i+1], H)
        
        # Save panorama
        output_path = OUTPUT_PATH / f"{scene_name}_panorama_{method}.png"
        cv2.imwrite(str(output_path), panorama)
        print(f"  Saved to: {output_path}")
        
        # Visualize
        plt.figure(figsize=(20, 10))
        plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
        plt.title(f"{scene_name} - Panorama ({method})")
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(OUTPUT_PATH / f"{scene_name}_panorama_{method}_viz.png", dpi=150, bbox_inches='tight')
        plt.show()
        
        return {
            'scene': scene_name,
            'method': method,
            'panorama': panorama,
            'statistics': statistics
        }
    
    return None

print("Scene processing function defined!")

In [ ]:
# Process all scenes
scenes = ['v_bird', 'v_boat', 'v_circus', 'v_graffiti', 'v_soldiers', 'v_weapons']
all_results = []

for scene in scenes:
    result = process_scene(scene, method='SIFT', num_images=6)
    if result:
        all_results.append(result)

print(f"\n{'='*60}")
print(f"Completed processing {len(all_results)} scenes")
print(f"{'='*60}")

## Feature Detector Comparison <a name="comparison"></a>

Compare SIFT, SURF, and ORB on a sample scene.

In [ ]:
def compare_detectors(scene_name: str = 'v_bird'):
    """Compare SIFT, SURF, and ORB detectors on a scene."""
    
    scene_path = PANORAMA_PATH / scene_name
    img1 = cv2.imread(str(scene_path / "1.png"))
    img2 = cv2.imread(str(scene_path / "2.png"))
    
    methods = ['SIFT', 'ORB']  # Skip SURF if not available
    comparison_data = []
    
    for method in methods:
        print(f"\n{'='*60}")
        print(f"Testing {method}")
        print(f"{'='*60}")
        
        try:
            start_time = time.time()
            
            # Extract features
            extractor = FeatureExtractor(method=method)
            kp1, desc1 = extractor.detect_and_compute(img1)
            kp2, desc2 = extractor.detect_and_compute(img2)
            
            # Match features
            matcher = FeatureMatcher(descriptor_type=extractor.descriptor_type, ratio_threshold=0.75)
            matches = matcher.match(desc1, desc2)
            
            # Estimate homography
            if len(matches) >= 4:
                src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
                dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])
                
                estimator = HomographyEstimator(ransac_iterations=1000, ransac_threshold=5.0)
                H, inlier_mask, inlier_ratio = estimator.estimate_homography_ransac(src_pts, dst_pts)
                
                errors = estimator.compute_reprojection_error(src_pts[inlier_mask], 
                                                             dst_pts[inlier_mask], H)
                avg_error = np.mean(errors)
            else:
                inlier_ratio = 0
                avg_error = float('inf')
            
            elapsed_time = time.time() - start_time
            
            data = {
                'Method': method,
                'Keypoints (img1)': len(kp1),
                'Keypoints (img2)': len(kp2),
                'Matches': len(matches),
                'Inliers': np.sum(inlier_mask) if len(matches) >= 4 else 0,
                'Inlier Ratio (%)': f"{inlier_ratio*100:.2f}",
                'Avg Reprojection Error': f"{avg_error:.3f}",
                'Runtime (s)': f"{elapsed_time:.3f}"
            }
            comparison_data.append(data)
            
            print(f"Keypoints: {len(kp1)}, {len(kp2)}")
            print(f"Matches: {len(matches)}")
            print(f"Inlier ratio: {inlier_ratio*100:.2f}%")
            print(f"Runtime: {elapsed_time:.3f}s")
            
        except Exception as e:
            print(f"Error with {method}: {e}")
    
    # Create comparison table
    import pandas as pd
    df = pd.DataFrame(comparison_data)
    print(f"\n{'='*60}")
    print("COMPARISON TABLE")
    print(f"{'='*60}")
    print(df.to_string(index=False))
    
    return df

# Run comparison
comparison_df = compare_detectors('v_bird')

## Augmented Reality Application <a name="ar"></a>

Implement the AR application that projects a source video onto a book cover in another video.

In [ ]:
class ARVideoProcessor:
    """
    Augmented Reality video processor for projecting source video onto planar target.
    """
    
    def __init__(self, reference_image_path: str, target_video_path: str, 
                 source_video_path: str, output_path: str):
        """
        Initialize AR video processor.
        
        Args:
            reference_image_path: Path to reference image (book cover)
            target_video_path: Path to target video (book.mov)
            source_video_path: Path to source video to project (ar_source.mov)
            output_path: Path to save output video
        """
        self.reference_image_path = reference_image_path
        self.target_video_path = target_video_path
        self.source_video_path = source_video_path
        self.output_path = output_path
        
        # Load reference image
        self.reference = cv2.imread(reference_image_path)
        if self.reference is None:
            raise ValueError(f"Failed to load reference image: {reference_image_path}")
        
        # Extract features from reference once
        self.extractor = FeatureExtractor(method='SIFT')
        self.ref_kp, self.ref_desc = self.extractor.detect_and_compute(self.reference)
        print(f"Reference image keypoints: {len(self.ref_kp)}")
        
        # Initialize matcher and estimator
        self.matcher = FeatureMatcher(descriptor_type='float', ratio_threshold=0.75)
        self.estimator = HomographyEstimator(ransac_iterations=1000, ransac_threshold=5.0)
        
        # Open videos
        self.target_cap = cv2.VideoCapture(target_video_path)
        self.source_cap = cv2.VideoCapture(source_video_path)
        
        if not self.target_cap.isOpened():
            raise ValueError(f"Failed to open target video: {target_video_path}")
        if not self.source_cap.isOpened():
            raise ValueError(f"Failed to open source video: {source_video_path}")
        
        # Get video properties
        self.target_fps = self.target_cap.get(cv2.CAP_PROP_FPS)
        self.target_width = int(self.target_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.target_height = int(self.target_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.target_frame_count = int(self.target_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        self.source_frame_count = int(self.source_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Target video: {self.target_width}x{self.target_height} @ {self.target_fps} fps, "
              f"{self.target_frame_count} frames")
        print(f"Source video: {self.source_frame_count} frames")
    
    def get_book_corners(self, frame: np.ndarray, H: np.ndarray) -> np.ndarray:
        """
        Get corners of the book in the frame using homography.
        
        Args:
            frame: Current frame
            H: Homography from reference to frame
            
        Returns:
            corners: 4x2 array of corner coordinates
        """
        h, w = self.reference.shape[:2]
        ref_corners = np.array([
            [0, 0],
            [w, 0],
            [w, h],
            [0, h]
        ], dtype=np.float32)
        
        # Transform corners
        ref_corners_h = np.hstack([ref_corners, np.ones((4, 1))])
        frame_corners_h = (H @ ref_corners_h.T).T
        frame_corners = frame_corners_h[:, :2] / frame_corners_h[:, 2:3]
        
        return frame_corners.astype(np.int32)
    
    def crop_and_warp_source(self, source_frame: np.ndarray, H: np.ndarray, 
                            output_size: Tuple[int, int]) -> np.ndarray:
        """
        Crop source frame to center and warp to match book perspective.
        
        Args:
            source_frame: Source video frame
            H: Homography from reference to target frame
            output_size: (width, height) of reference
            
        Returns:
            warped: Warped source frame
        """
        src_h, src_w = source_frame.shape[:2]
        ref_h, ref_w = output_size
        
        # Calculate aspect ratios
        src_aspect = src_w / src_h
        ref_aspect = ref_w / ref_h
        
        # Crop source to match reference aspect ratio (center crop)
        if src_aspect > ref_aspect:
            # Source is wider - crop width
            new_w = int(src_h * ref_aspect)
            x_offset = (src_w - new_w) // 2
            cropped = source_frame[:, x_offset:x_offset + new_w]
        else:
            # Source is taller - crop height
            new_h = int(src_w / ref_aspect)
            y_offset = (src_h - new_h) // 2
            cropped = source_frame[y_offset:y_offset + new_h, :]
        
        # Resize to reference size
        resized = cv2.resize(cropped, (ref_w, ref_h))
        
        # Warp using homography
        warped = cv2.warpPerspective(resized, H, (self.target_width, self.target_height))
        
        return warped
    
    def composite_frame(self, target_frame: np.ndarray, warped_source: np.ndarray, 
                       mask: np.ndarray) -> np.ndarray:
        """
        Composite warped source onto target frame.
        
        Args:
            target_frame: Target video frame
            warped_source: Warped source frame
            mask: Binary mask of valid region
            
        Returns:
            result: Composited frame
        """
        result = target_frame.copy()
        
        # Apply warped source where mask is non-zero
        mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) if len(mask.shape) == 2 else mask
        result = np.where(mask_3ch > 0, warped_source, result)
        
        return result
    
    def process_video(self, frame_skip: int = 1, max_frames: Optional[int] = None):
        """
        Process videos and create AR output.
        
        Args:
            frame_skip: Process every Nth frame (1 = all frames)
            max_frames: Maximum number of frames to process (None = all)
        """
        # Setup video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path, fourcc, self.target_fps / frame_skip,
                             (self.target_width, self.target_height))
        
        if not out.isOpened():
            raise ValueError(f"Failed to open video writer: {self.output_path}")
        
        frame_idx = 0
        processed_count = 0
        failed_count = 0
        
        # Determine number of frames to process
        total_frames = min(self.target_frame_count, self.source_frame_count)
        if max_frames:
            total_frames = min(total_frames, max_frames)
        
        print(f"\nProcessing {total_frames} frames (skip={frame_skip})...")
        
        while True:
            # Read target frame
            ret_target, target_frame = self.target_cap.read()
            if not ret_target:
                break
            
            # Read source frame (loop if needed)
            source_idx = frame_idx % self.source_frame_count
            self.source_cap.set(cv2.CAP_PROP_POS_FRAMES, source_idx)
            ret_source, source_frame = self.source_cap.read()
            if not ret_source:
                break
            
            # Skip frames if needed
            if frame_idx % frame_skip != 0:
                frame_idx += 1
                continue
            
            # Stop if max frames reached
            if max_frames and processed_count >= max_frames:
                break
            
            try:
                # Detect and match features
                frame_kp, frame_desc = self.extractor.detect_and_compute(target_frame)
                matches = self.matcher.match(self.ref_desc, frame_desc)
                
                if len(matches) < 4:
                    print(f"Frame {frame_idx}: Insufficient matches ({len(matches)})")
                    failed_count += 1
                    out.write(target_frame)  # Write original frame
                    frame_idx += 1
                    processed_count += 1
                    continue
                
                # Estimate homography
                src_pts = np.float32([self.ref_kp[m.queryIdx].pt for m in matches])
                dst_pts = np.float32([frame_kp[m.trainIdx].pt for m in matches])
                
                H, inlier_mask, inlier_ratio = self.estimator.estimate_homography_ransac(src_pts, dst_pts)
                
                if inlier_ratio < 0.1:  # Too few inliers
                    print(f"Frame {frame_idx}: Low inlier ratio ({inlier_ratio*100:.1f}%)")
                    failed_count += 1
                    out.write(target_frame)
                    frame_idx += 1
                    processed_count += 1
                    continue
                
                # Warp source frame
                ref_h, ref_w = self.reference.shape[:2]
                warped_source = self.crop_and_warp_source(source_frame, H, (ref_w, ref_h))
                
                # Create mask for valid region
                mask = (np.sum(warped_source, axis=2) > 0).astype(np.uint8) * 255
                
                # Composite
                result = self.composite_frame(target_frame, warped_source, mask)
                
                # Write frame
                out.write(result)
                
                if processed_count % 10 == 0:
                    print(f"Processed frame {frame_idx}/{total_frames} "
                          f"({processed_count} written, {failed_count} failed, "
                          f"inliers: {inlier_ratio*100:.1f}%)")
                
            except Exception as e:
                print(f"Frame {frame_idx} error: {e}")
                failed_count += 1
                out.write(target_frame)
            
            frame_idx += 1
            processed_count += 1
        
        # Cleanup
        self.target_cap.release()
        self.source_cap.release()
        out.release()
        
        print(f"\n{'='*60}")
        print(f"AR video processing complete!")
        print(f"Total frames processed: {processed_count}")
        print(f"Failed frames: {failed_count}")
        print(f"Output saved to: {self.output_path}")
        print(f"{'='*60}")

print("ARVideoProcessor class defined!")

In [ ]:
# Process AR video
# NOTE: This is computationally intensive. Start with a small number of frames for testing.

reference_img = str(AR_PATH / "cv_cover.jpg")
target_video = str(AR_PATH / "book.mov")
source_video = str(AR_PATH / "ar_source.mov")
output_video = str(OUTPUT_PATH / "ar_dynamic_result.mp4")

print("Initializing AR video processor...")
print("NOTE: Processing full video may take several hours.")
print("Starting with first 30 frames for testing...")

ar_processor = ARVideoProcessor(reference_img, target_video, source_video, output_video)

# Process video (start with limited frames for testing)
# Change max_frames=None to process entire video
ar_processor.process_video(frame_skip=1, max_frames=30)

print("\nAR video processing test complete!")
print("To process the full video, set max_frames=None in the process_video() call above.")

## Summary and Statistics <a name="summary"></a>

Generate summary statistics and comparison tables for the report.

In [ ]:
# Create summary statistics table
print("="*80)
print("PANORAMA STITCHING SUMMARY")
print("="*80)

for result in all_results:
    print(f"\n{result['scene']} ({result['method']}):")
    print("-" * 60)
    for stats in result['statistics']:
        print(f"  {stats['image_pair']:>6}: "
              f"Matches: {stats['matches']:>4}, "
              f"Inliers: {stats['inliers']:>4} ({stats['inlier_ratio']*100:>5.1f}%), "
              f"Error: {stats['avg_reprojection_error']:>6.3f}px")

print("\n" + "="*80)
print("ASSIGNMENT COMPLETION CHECKLIST")
print("="*80)
print("✓ Part 1: Feature Extraction (SIFT, SURF, ORB) - Implemented")
print("✓ Part 2: Feature Matching with Lowe's ratio test - Implemented")
print("✓ Part 3: DLT + RANSAC Homography Estimation - Implemented")
print("✓ Part 4: Image Warping and Panorama Stitching - Implemented")
print("✓ AR Application: Video projection on moving target - Implemented")
print("\n" + "="*80)

## Instructions and Next Steps

### What has been implemented:

1. **Feature Extraction (Part 1)**: Complete implementation of SIFT, SURF, and ORB detectors with visualization
2. **Feature Matching (Part 2)**: k-NN matching with Lowe's ratio test and optional cross-checking
3. **Homography Estimation (Part 3)**: Manual DLT algorithm with point normalization and RANSAC
4. **Panorama Stitching (Part 4)**: Image warping with multiple blending methods (copy, average, linear feathering)
5. **AR Application**: Video projection framework with per-frame homography estimation

### To complete the assignment:

1. **Run all cells sequentially** - Start from the top and execute each cell
2. **Install required packages** if needed: `opencv-contrib-python` for SURF (optional), `scipy` for distance transforms
3. **Process all scenes** - The notebook will create panoramas for all 6 scenes
4. **AR Video Processing** - Currently set to process 30 frames for testing. To process full video:
   - Change `max_frames=30` to `max_frames=None` in the AR processing cell
   - **Warning**: This may take several hours depending on your hardware
   - Consider using `frame_skip=2` or `frame_skip=3` to speed up processing
5. **Upload videos** - Upload `ar_dynamic_result.mp4` to Google Drive/OneDrive and include link in report

### Files Generated:

- `panorama_results/[scene]_panorama_SIFT.png` - Panorama for each scene
- `panorama_results/visualizations/` - Keypoint and matching visualizations
- `panorama_results/ar_dynamic_result.mp4` - AR video output

### Report Structure (as per assignment):

1. **Overview**: Summarize the pipeline and objectives
2. **Dataset & Setup**: Describe panorama_dataset and ar_dataset
3. **Methods**: 
   - Explain SIFT/SURF/ORB choices and properties
   - Describe matching strategy (k-NN, ratio test)
   - Detail DLT derivation and RANSAC parameters
   - Explain warping and blending techniques
   - Describe AR per-frame processing
4. **Results**: Include all visualizations generated by this notebook
5. **Discussion**: Analyze when the pipeline works well vs. fails
6. **Reproducibility**: List all parameters used

### Key Parameters Used:

- **Lowe's ratio threshold**: 0.75
- **RANSAC iterations**: 2000 (panorama), 1000 (AR)
- **RANSAC inlier threshold**: 5.0 pixels
- **Point normalization**: Enabled (improves numerical stability)
- **Blending method**: Linear (feathering) for smooth transitions
- **ORB keypoints**: 5000 maximum
- **Random seed**: 42 (for reproducibility)

### Tips:

- Check `OUTPUT_PATH` for all generated files
- Use comparison table to analyze SIFT vs ORB performance
- Save intermediate results frequently
- For AR video, verify first few frames before processing full video
- Consider optical flow tracking (cv2.calcOpticalFlowPyrLK) for more stable AR if needed